In [ ]:
from functools import partial

from sepal_ui import sepalwidgets as sw
from sepal_ui import mapping as sm
import ipyvuetify as v

from scripts import run
from utils import messages as ms
from utils import parameters as pm

In [ ]:
#use a class to define your input and output in order to have mutable variables
class Gfc_viz_io:
    def __init__(self):
        #set up your inputs
        self.threshold = 30
        #set up output
        self.visualization = False
        
gfc_viz_io = Gfc_viz_io()

In [ ]:
#create a map
gfc_m = sm.SepalMap()
gfc_m.add_legend(legend_keys=pm.getMyLabel(), legend_colors=pm.getColorPalette(), position='topleft')

In [ ]:
#create the output alert 
gfc_viz_output = sw.Alert()#.add_msg(ms.GFC_MAP)

#create the button that start your process
gfc_viz_btn = sw.Btn('Update map', icon='mdi-check')

#create the widgets following ipyvuetify requirements 
gfc_viz_slider = v.Slider(label= 'Threshold', class_="mt-5", thumb_label='always', v_model=30)
gfc_viz_output.bind(gfc_viz_slider, gfc_viz_io, 'threshold')


#create a process tile
id_ = "gfc_map_widget"
title = 'GFC visualization'

#create a custom tile to display the vizualization map without scrolling down 
gfc_viz_inputs = v.Layout(
    _metadata={'mount-id': 'data-input'},
    class_="pa-5",
    row=True,
    align_center=True, 
    children=[
        v.Flex(xs12=True, children=[gfc_viz_slider]),
        v.Flex(xs12=True, children=[gfc_viz_btn]),
        v.Flex(xs12=True, children=[gfc_viz_output]),
    ]
)

gfc_viz_inputs = v.Layout(
    row=True,
    xs12=True,
    children=[
        v.Flex(xs12=True, md6=True, children=[gfc_viz_inputs]),
        v.Flex(class_="pa-5", xs12=True, md6=True, children=[gfc_m])
    ]
) 

gfc_viz = sw.Tile(
    id_= id_,
    title= title,
    inputs = [gfc_viz_inputs]
)

#bind the button to the process by writing a custom function
def process_start(widget, event, data, output, m):
    
    asset = gfc_aoi_io.assetId
    threshold = gfc_viz_io.threshold
    visualization = gfc_viz_io.visualization
    
    #toggle the loading button
    widget.toggle_loading()
    
    #check inputs
    if not output.check_input(asset, ms.NO_AOI): return widget.toggle_loading()
    if not output.check_input(threshold, ms.NO_THRESHOLD): return widget.toggle_loading()
    
    try:
        #display the results 
        run.displayGfcMap(asset, threshold, m, visualization, output)
    
        #validate the visualization process
        setattr(gfc_viz_io, 'visualization', True)
    
    except Exception as e: 
        output.add_live_msg(str(e), 'error')
        
    #toggle the loading button
    widget.toggle_loading()
    
    return 

gfc_viz_btn.on_event('click', partial(
    process_start, 
    output=gfc_viz_output,
    m=gfc_m,
))

In [ ]:
#this tiles will only be displayed if you launch voila from this file 
gfc_viz